## 0. 환경 구성

### 1) 라이브러리 설치

In [4]:
%pip install -q langchain langchain-openai langchain_community tiktoken chromadb onnxruntime

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%pip show onnxruntime

Name: onnxruntime
Version: 1.20.1
Summary: ONNX Runtime is a runtime accelerator for Machine Learning models
Home-page: https://onnxruntime.ai
Author: Microsoft Corporation
Author-email: onnxruntime@microsoft.com
License: MIT License
Location: c:\Users\vega2\anaconda3\Lib\site-packages
Requires: coloredlogs, flatbuffers, numpy, packaging, protobuf, sympy
Required-by: chromadb
Note: you may need to restart the kernel to use updated packages.


### 2) OpenAI 인증키 설정
https://openai.com/

In [2]:
from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

True

#### RAG 파이프 라인
* Load Data - Text Split - Indexing - Retrieval - Generation

In [4]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

# 1. Load Data
loader = TextLoader("data/knowledge.txt", encoding="utf-8")
documents = loader.load()

# 2️. Text Split
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(documents)

# 3️. Indexing (벡터 저장)
vectorstore = Chroma.from_documents(split_docs, \
                                    OpenAIEmbeddings())

# 4️. Retrieval (유사 문서 검색)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
# **질문(쿼리)**에 대해 유사한 문서를 검색하는 역할
retrieved_docs = retriever.invoke("Transformer 모델은 어떻게 동작하나요?")

# 5️. Generation (LLM 응답 생성)
llm = ChatOpenAI(model="gpt-3.5-turbo")
context = "\n\n".join([doc.page_content for doc in retrieved_docs])
# LLM에게 단순한 질문만 하는 것이 아니라 **유사 문서 검색 내용**도 함께 제공하여
# LLM이 문서의 정보를 참고하여 더 정확하고 관련성 높은 답변을 생성할 수 있습니다.
response = llm.invoke(f"Transformer 모델은 어떻게 동작하나요? 관련 정보: {context}")

print(response)


content='\n데이터를 벡터 형식으로 저장하여 유사성을 계산하고 효과적으로 검색할 수 있음\n벡터 간의 거리를 계산하여 유사한 데이터를 찾음\n대량의 데이터를 처리하고 빠르게 검색할 수 있는 장점이 있음\n예시:\nWord2Vec, Doc2Vec, Faiss와 같은 벡터 데이터베이스 라이브러리가 활용됨\n\n연관 키워드:\n벡터화, 유사도 계산, 검색, 머신러닝, 대량 데이터 처리' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 164, 'prompt_tokens': 627, 'total_tokens': 791, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-6f081d9b-2470-45d3-a6f8-b9c8ac3704c6-0' usage_metadata={'input_tokens': 627, 'output_tokens': 164, 'total_tokens': 791, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
